In [1]:
%reload_ext autoreload
%autoreload 1

In [2]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

curruser = os.environ.get('USER')

# sys.path.insert(0, '/opt/workspace/{user}/system/support_library/'.format(user=curruser))
# sys.path.insert(0, '/opt/workspace/{user}/libs/'.format(user=curruser))
# sys.path.insert(0, '/opt/workspace/{user}/system/labdata/lib/'.format(user=curruser))

# sys.path.insert(0, '/opt/workspace/{}/notebooks/Clickstream_Analytics/scr'.format(curruser))
sys.path.insert(0, '/opt/workspace/{user}/notebooks/support_library/'.format(user=curruser)) 
sys.path.insert(0, '/opt/workspace/{user}/libs/python3.5/site-packages/'.format(user=curruser))
sys.path.insert(0, '/opt/workspace/{user}/notebooks/labdata/lib/'.format(user=curruser))

# sys.path.insert(0, '/home/{}/notebooks/support_library/'.format(curruser)) 
# sys.path.insert(0, '/home/{}/python35-libs/lib/python3.5/site-packages/'.format(curruser))
# sys.path.insert(0, '/home/{}/notebooks/labdata/lib/'.format(curruser))

# from tqdm import tqdm
# from tqdm._tqdm_notebook import tqdm_notebook
# tqdm_notebook.pandas()

from pathlib import Path
import pandas as pd
import numpy as np
import re 
import joblib
import subprocess

# pd.set_option('display.float_format', lambda x: '%.3f' % x)
# pd.set_option('display.max_colwidth', -1)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

from spark_connector import SparkConnector
from sparkdb_loader import spark
# from spark_helper import SparkHelper
from connector import OracleDB
import pyspark
from pyspark import SparkContext, SparkConf, HiveContext
from pyspark.sql.functions import udf 
from pyspark.sql import functions as f
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql.dataframe import DataFrame
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import StringIndexer

# import loader as load

# from corpora_process import utils
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [3]:
src_path = os.getcwd() + '/src/'
sys.path.insert(0, src_path) 

from urlprocessing import ParseURL
from collections import OrderedDict
from CRMDictProds import *
from datetime import datetime

In [4]:
def show(self, n=10):
    return self.limit(n).toPandas()

pyspark.sql.dataframe.DataFrame.show = show

In [5]:
%%time
sp = spark(schema='sklod_external_google_analytics',
           queue = 'ektov1-av_ca-sbrf-ru',
           process_label="anthony_",
           numofinstances=20, 
           numofcores=8,
           dynamic_alloc=False,
           kerberos_auth=False
          ) 
print(sp.sc.version)
hive = sp.sql

2.4.0.cloudera2
CPU times: user 55 ms, sys: 53.1 ms, total: 108 ms
Wall time: 33.7 s


In [ ]:
sdf = hive.table("sklod_external_google_analytics.visit")
print("\n".join(["{}: {}".format(col.upper(),_type) for col, _type in sdf.dtypes]))

In [7]:
refdate0 = '2020-07-01'
refdate1 = '2020-12-01'

In [8]:
sql = \
'''
    select 
    cid, 
    sessionId, 
    visitNumber,
    hitType, 
    hitPagePath, 
    hitPageTitle, 
    sessionStartTime, 
    hitNumber, 
    hitTime, 
    eventCategory, 
    eventAction, 
    eventLabel
from sklod_external_google_analytics.visit
where 
    hitPageHostName like '%www.sberbank.ru%'
    and hitPagePath like '%s_m_business%'
    and cid <> '-.-'
    --and hitPageHostName not in ('sbi.sberbank.ru', 'localhost') 
    --and hitType = 'PAGE' 
    and (timestamp(sessionDate) >= timestamp('{}') and timestamp(sessionDate) < timestamp('{}'))
'''.format(refdate0,refdate1)

In [9]:
gadf_trimmed = hive.sql(sql)

In [10]:
gadf_trimmed = gadf_trimmed.withColumn('sessionTime', 
                                       f.from_unixtime(f.col('sessionStartTime')+f.round(f.col('hitTime')/1000))
                                      )
gadf_trimmed = gadf_trimmed.orderBy(f.col('sessionTime').asc())

In [11]:
gadf_trimmed.cache()

DataFrame[cid: string, sessionId: bigint, visitNumber: bigint, hitType: string, hitPagePath: string, hitPageTitle: string, sessionStartTime: bigint, hitNumber: bigint, hitTime: bigint, eventCategory: string, eventAction: string, eventLabel: string, sessionTime: string]

In [12]:
print("\n".join(["{}: {}".format(col.upper(),_type) for col, _type in gadf_trimmed.dtypes]))

CID: string
SESSIONID: bigint
VISITNUMBER: bigint
HITTYPE: string
HITPAGEPATH: string
HITPAGETITLE: string
SESSIONSTARTTIME: bigint
HITNUMBER: bigint
HITTIME: bigint
EVENTCATEGORY: string
EVENTACTION: string
EVENTLABEL: string
SESSIONTIME: string


In [13]:
gadf_trimmed.count()

175082892

In [14]:
gadf_trimmed = gadf_trimmed.drop("visitNumber", "hitTime")

In [15]:
w1=Window.orderBy('cid', 'sessionId', 'row_num')
w2=Window.partitionBy('cid', 'sessionId').orderBy('sessionId')

gadf_trimmed = gadf_trimmed.withColumn('row_num', f.row_number().over(w2))\
                           .withColumn('marker_col', f.when(f.col('hitType')=="PAGE",f.col('row_num')).otherwise(None))\
                           .withColumn('last', f.last('marker_col',True).over(Window.partitionBy('cid', 'sessionId')\
                                                                                    .orderBy('row_num').rowsBetween(Window.unboundedPreceding, Window.currentRow)))\

gadf_trimmed=\
gadf_trimmed.withColumn('visitNumber', f.when(((f.lead(f.col('hitType'),count=1).over(w1)=='PAGE')&\
                                                    (f.lead(f.col('hitType'),count=2).over(w1)=='PAGE')
                                                   )&(f.col('hitType')!='EVENT')
                                                   ,f.lead(f.col('last'),count=2).over(w1)
                                                  )\
                                             .when(((f.lead(f.col('hitType'),count=1).over(w1)=='PAGE')&\
                                                    (f.lead(f.col('hitType'),count=2).over(w1)!='PAGE')
                                                   )&(f.col('hitType')!='EVENT')
                                                   ,f.lead(f.col('last'),count=1).over(w1)
                                                  )\
                                             .otherwise(f.col('last'))
                       )

gadf_trimmed = gadf_trimmed.drop(*["row_num", "marker_col", "last", ])

In [28]:
# res = \
# gadf_trimmed.filter(((f.col("hitPagePath") == "/ru/s_m_business/pro_busines")|
#                      (f.col("hitPagePath") == "/ru/s_m_business/pro_business/main/")
#                     )&
#                     (f.col("hitType")=="EVENT")&
#                     (f.col('eventAction').like('%informing_leads%'))&
#                     (f.col('eventLabel').like("%scroll 30 percent%")|
#                      f.col('eventLabel').like("%scroll 50 percent%")|
#                      f.col('eventLabel').like("%scroll 75 percent%")|
#                      f.col('eventLabel').like("%sec on page%")
#                     )&
#                     (~(f.col('eventAction').like('%show_banner%'))&
#                      ~(f.col('eventAction').like('%SBBOL%'))&
#                      ~(f.col('eventAction').like('%show_ear%'))&
#                      ~(f.col('eventAction').like('%cookie_set%'))&
#                      ~(f.col('eventAction').like('%close_segments%'))
#                     )&
#                     (~f.lower(f.col('eventLabel')).like("%сбербанк%бизнес%онлайн%")&
#                      ~f.lower(f.col('eventLabel')).like("%личный%кабинет%")
#                     )
#                     #(f.col("sessionTime") < pd.to_datetime(pd.datetime(2020,9,24)))
#                    ).select(f.col("eventLabel")).distinct()
                            
res.show(100)                            

,eventLabel
0,30 sec on page
1,scroll 50 percent
2,scroll 75 percent


0

In [16]:
gadf_trimmed.registerTempTable("tmp_visit")
#hive.cacheTable("tmp_visit")
hive.sql("drop table if exists {}.{} purge".format("sbx_team_digitcamp", "recsys_ga_visit_fe_part"))
subprocess.call(['hdfs', 'dfs', '-rm', '-r', '-f', '-skipTrash', "hdfs://clsklsbx/user/team/team_digitcamp/hive/{}".format("recsys_ga_visit_fe_part")])
hive.sql("create table {}.{} select * from tmp_visit".format("sbx_team_digitcamp", "recsys_ga_visit_fe_part"))

DataFrame[]

# FE

## Fetching saved data

In [17]:
res = hive.table("{}.{}".format("sbx_team_digitcamp", "recsys_ga_visit_fe_part"))

In [18]:
res.select(f.max("sessionTime")).show()

,max(sessionTime)
0,2020-11-14 23:59:59


### UDF

In [19]:
def checkhitPageTitle(dctofrefwords, title, path):
#     pattern2= re.compile(r'([a-zA-Z=\-\_\&,.#\/0-9]+)(?:\?.*)*')
    finout = None
    for key, wordlst in dctofrefwords.items():
        for reword in wordlst:
            wpatt = re.compile(reword)
            if title is not None:
                res = wpatt.findall(title.lower())
                if len(res)!=0:
                    if path is not None:
    #                     out=pattern2.findall(path)
                        finout = key
                        break
                    else:
                        None
                else:
                     None
            else:
                None

        if finout is not None:
            break

    return finout

def udf_checkhitPageTitle (lst):
    return f.udf(lambda t,p: checkhitPageTitle(lst,t,p))

In [20]:
sp.sc.addPyFile('./src/urlprocessing.py')
GetUrl = sp.sc.broadcast(ParseURL())

get_url_domain_udf = f.udf(GetUrl.value.get_url_domain, StringType())
get_url_path_udf   = f.udf(GetUrl.value.get_url_path, StringType())

parse = AllParseAndRegExp()

In [21]:
@udf(StringType())
def get_hitText(ec,ea,el):
    extractedres = []
    patt = "(^[а-яА-Яa-zA-Z -,.]+)(?:_[a-zA-Z0-9 _]+)$"
    findpatt = re.compile(patt)
    if el is not None:
        extractedres = findpatt.findall(el)
        
    if (ec is not None) and ('category' in ec.lower()):
        return re.sub("\.","",ec.split("category.")[-1])
    elif (ea is not None) and ("click_any" in ea.lower()) and (len(extractedres) > 0):
        return extractedres[0]
    else:
        return None
    
@udf(StringType())
def get_hitElemType(ea,el):
    extractedres = []
    patt = "(?:[а-яА-Яa-zA-Z -,.]+)(_[a-zA-Z0-9 _-]+)"
    findpatt = re.compile(patt)
    if el is not None:
        extractedres = findpatt.findall(el)
        
    if (ea is not None) and ("click_any" in ea.lower()) and (len(extractedres) > 0):
        return extractedres[0]
    else:
        return None
    
    
@udf(StringType())
def get_hitTextStyle(ea,el):
    extractedres = []
    patt = "(?:.*_stk-reset.*)__style_([a-zA-Z_-]+)"
    findpatt = re.compile(patt)
    if el is not None:
        extractedres = findpatt.findall(el)
        
    if (ea is not None) and ("click_any" in ea.lower()) and (len(extractedres) > 0):
        return extractedres[0]
    else:
        return None

In [22]:
res = res.withColumn("url_path",      get_url_path_udf("hitPagePath")) \
         .withColumn("hitText",       get_hitText("eventCategory", "eventAction", "eventLabel")) \
         .withColumn("hitElemType",   get_hitElemType("eventAction", "eventLabel")) \
         .withColumn("hitTextStyle",  get_hitTextStyle("eventAction", "eventLabel")) \
         .withColumn('prodExtractedFromTitle', udf_checkhitPageTitle(parse.dctofrefwords)('hitPageTitle', 'url_path'))

In [143]:
res.filter(f.col("hitElemType").like('%_stk%') & 
           (~f.isnull(f.col("hitTextStyle")))
          ).select("eventLabel", "hitElemType", "hitTextStyle").show(30)

,eventLabel,hitElemType,hitTextStyle
0,Копирование_stk-reset stk-theme_28333__style_m...,_stk-reset stk-theme_28333__style_medium_header,medium_header
1,Что такое альтернативная стоимость_stk-reset s...,_stk-reset stk-theme_28333__style_large_header,large_header
2,"Запас финансовой прочности: как узнать, стабил...",_stk-reset stk-theme_28333__style_font_style-1...,font_style-
3,"Запас финансовой прочности: как узнать, стабил...",_stk-reset stk-theme_28333__style_font_style-1...,font_style-
4,"Запас финансовой прочности: как узнать, стабил...",_stk-reset stk-theme_28333__style_font_style-1...,font_style-
5,"Запас финансовой прочности: как узнать, стабил...",_stk-reset stk-theme_28333__style_font_style-1...,font_style-
6,1_stk-reset stk-theme_35869__style_medium_head...,_35869__style_medium_header stk-theme_35869__c...,medium_header
7,1_stk-reset stk-theme_35869__style_medium_head...,_35869__style_medium_header stk-theme_35869__c...,medium_header
8,Онлайн-кассы можно поделить на несколько катег...,_28333__style_medium_header stk-theme_28333__c...,medium_header
9,Всё через онлайн-кассу (ККТ)_stk-reset stk-the...,_stk-reset stk-theme_35869__style_large_header,large_header


In [23]:
## 
w=Window.partitionBy('cid').orderBy('sessionId') 

# res = gadf_trimmed #.filter(f.col("sessionTime") < pd.to_datetime(pd.datetime(2020,9,24)))

res = res.withColumn('countPageHit',f.sum(f.when(
                                                            ((f.col("hitPagePath") == "/ru/s_m_business/pro_business")|
                                                             (f.col("hitPagePath") == "/ru/s_m_business/pro_business/main/")
                                                            )&
                                                            (f.col("hitType")=="PAGE")
                                                            ,1
                                                        ).otherwise(0)).over(w)
                    )

res = res.withColumn('countEventHit',f.sum(f.when(
                                                            ((f.col("hitPagePath") == "/ru/s_m_business/pro_business")|
                                                             (f.col("hitPagePath") == "/ru/s_m_business/pro_business/main/")
                                                            )&
                                                            (f.col("hitType")=="EVENT")
                                                            ,1
                                                        ).otherwise(0)).over(w)
                    )

res = res.withColumn("interestedEvents", f.when(
                                                    (~(f.col('eventAction').like('%show_banner%'))&
                                                     ~(f.col('eventAction').like('%SBBOL%'))&
                                                     ~(f.col('eventAction').like('%show_ear%'))&
                                                     ~(f.col('eventAction').like('%cookie_set%'))&
                                                     ~(f.col('eventAction').like('%close_segments%'))
                                                    )&
                                                    (~f.lower(f.col('eventLabel')).like("%сбербанк%бизнес%онлайн%")&
                                                     ~f.lower(f.col('eventLabel')).like("%личный%кабинет%")&
                                                     (f.regexp_extract(f.lower(f.col('eventLabel')),
                                                                              "(.*vhod.*|.*войти.*|.*вход.*)", 1) == "")
                                                    ) 
                                                   ,1
                                                ).otherwise(0)
                    )

In [24]:

res = res.withColumn('sumXXSecOnPage', f.sum(f.when(
                                                        (f.col("hitType")=="EVENT")&
                                                        (f.col("eventCategory").like("%SITE_Corporate_pro_business%"))& 
                                                        (f.col('eventAction').like('%informing_leads%'))&
                                                        (
                                                         f.col('eventLabel').like("%sec on page%")
                                                        )                                                        
                                                       ,1
                                                      ).otherwise(0)).over(w)
                    )

res = res.withColumn('sumScrollOnPage', f.sum(f.when(
                                                        (f.col("hitType")=="EVENT")&
                                                        (f.col("eventCategory").like("%SITE_Corporate_pro_business%"))& 
                                                        (f.col('eventAction').like('%informing_leads%'))&
                                                        (f.col('eventLabel').like("%scroll%percent%")
                                                        )                                                        
                                                       ,1
                                                     ).otherwise(0)).over(w)
                    )

In [25]:
w=Window.partitionBy('cid').orderBy('sessionId') 

patt = "(^[а-яА-Яa-zA-Z -,.]+)(?:__[a-zA-Z0-9]+)"
res = \
res.withColumn("articleTitles", f.when(
                                       (f.col("eventCategory").like("SITE_Corporate_pro_business%")) &
                                       (f.col('eventAction').like('%click%')) &
                                       (~f.col("eventLabel").like("%undefined%")) &
                                       (f.length(f.regexp_extract(f.col("eventLabel"), patt, 1)) > 0), 
                                       f.regexp_extract(f.col("eventLabel"), patt, 1)
                                      ).otherwise(None)
              )

res = res.withColumn('articleClick', f.when(~f.isnull(f.col("articleTitles")),1).otherwise(0))

res = res.withColumn('sumArticleClick', f.sum("articleClick").over(w)
                    )

patt2 = '(?<=SITE_Corporate_pro_business\.)[a-zA-Z\_\-]+(?=\.$)'

res = res.withColumn('getCategory', f.regexp_extract(f.col('hitPagePath'), r'(?<=/category/)\w+', 1))\
         .withColumn("getArticleName", f.regexp_extract(f.col("eventCategory"), patt2, 1))

res = res.withColumn('sessionRead', f.when(
                                            (f.col("interestedEvents") == 1) &
                                            (
                                             (f.col("sumXXSecOnPage") > 0)|
                                             (f.col("sumScrollOnPage") > 0)
                                            )&
                                            (f.col("countPageHit") > 0)&
                                            (f.col("sumArticleClick") > 0)
                                            ,1
                                           ).otherwise(0)
                    )

patt3 = "(^[а-яА-Яa-zA-Z =\&,.\:#\/0-9]+)(?:__[a-zA-Z0-9]+)"
res = \
res.withColumn("extractedTextFromClick",   f.regexp_replace(
                                                    f.regexp_extract(f.col('eventLabel'), patt3, 1),
                                                    "(^\\s+$|^\\s*[a-zA-Zа-яА-Я]{1,2}\\s*$)", "",
                                                  )
              )\
   .withColumn("extractedEvents",    f.regexp_extract(f.col('eventLabel'),"(^[a-zA-Z0-9\_\- ]+$)",1))
#    .withColumn("extractedArteFacts", f.regexp_extract(f.col('eventLabel'),"__([a-zA-Z0-9\_\- ]+)",1)) 


patt4 = "(^[а-яА-Яa-zA-Z \&,.\:#0-9]+)"
res = \
res.withColumn("extractedText",   f.regexp_replace(
                                                    f.regexp_extract(f.col('eventLabel'), patt4, 1),
                                                    "(^\\s+$|^\\s*[a-zA-Zа-яА-Я]{1,2}\\s*$)", "",
                                                  )
              )\


w2=Window.partitionBy('cid',"sessionId").orderBy('hitPagePath') 
res = \
res.withColumn("concatText", f.trim(f.concat_ws(" ", f.collect_list(f.col("extractedText")).over(w2))))

In [26]:
res = res.withColumn('firstPageHit', f.min(f.when((f.col('hitType')=='PAGE') &
                                                  (f.col("eventCategory").like("SITE_Corporate_pro_business%")), 
                                                  f.col('sessionTime')).otherwise(None)
                                          ).over(Window.partitionBy('cid','sessionId','visitNumber')))

res = res.withColumn('firstPageHitUrl', f.first(f.when((f.col('hitType')=='PAGE')&
                                                       (f.col("eventCategory").like("SITE_Corporate_pro_business%")) &
                                                       (f.col('sessionTime') == f.col('firstPageHit')), 
                                                        f.col('hitPagePath')).otherwise(None)
                                               ).over(Window.partitionBy('cid','sessionId','visitNumber')))

res = res.withColumn('lastPageHit',  f.max(f.when((f.col('hitType')=='PAGE')&
                                                  (f.col("eventCategory").like("SITE_Corporate_pro_business%")), 
                                                   f.col('sessionTime')
                                                 ).otherwise(None)
                                          ).over(Window.partitionBy('cid','sessionId','visitNumber')))

res = res.withColumn('firstEventHit', f.min(f.when((f.col('hitType')=='EVENT') &
                                                   (f.col("eventCategory").like("SITE_Corporate_pro_business%")), 
                                                   f.col('sessionTime')).otherwise(None)
                                           ).over(Window.partitionBy('cid','sessionId','visitNumber')))

res = res.withColumn('lastEventHit',  f.max(f.when((f.col('hitType')=='EVENT')&
                                                   (f.col("eventCategory").like("SITE_Corporate_pro_business%")), 
                                                    f.col('sessionTime')).otherwise(None)
                                           ).over(Window.partitionBy('cid','sessionId','visitNumber')))

res = res.withColumn('firstArticleClick', f.min(f.when(f.col('articleClick')==True, 
                                                       f.col('sessionTime')).otherwise(None)
                                               ).over(Window.partitionBy('cid','sessionId','visitNumber')))

res = res.withColumn('lastArticleClick',  f.max(f.when((f.col('articleClick')==True) , 
                                                        f.col('sessionTime')).otherwise(None)
                                               ).over(Window.partitionBy('cid','sessionId','visitNumber')))

res = res.withColumn('diffClickFirstPageHit', f.when(f.col('articleClick')==True, 
                                                 f.abs(f.unix_timestamp('sessionTime') -
                                                       f.unix_timestamp('firstPageHit')
                                                      ).cast(IntegerType())
                                           ).otherwise(0)
                    )

In [27]:
res.cache()

DataFrame[cid: string, sessionId: bigint, hitType: string, hitPagePath: string, hitPageTitle: string, sessionStartTime: bigint, hitNumber: bigint, eventCategory: string, eventAction: string, eventLabel: string, sessionTime: string, visitNumber: int, url_path: string, hitText: string, hitElemType: string, hitTextStyle: string, prodExtractedFromTitle: string, countPageHit: bigint, countEventHit: bigint, interestedEvents: int, sumXXSecOnPage: bigint, sumScrollOnPage: bigint, articleTitles: string, articleClick: int, sumArticleClick: bigint, getCategory: string, getArticleName: string, sessionRead: int, extractedTextFromClick: string, extractedEvents: string, extractedText: string, concatText: string, firstPageHit: string, firstPageHitUrl: string, lastPageHit: string, firstEventHit: string, lastEventHit: string, firstArticleClick: string, lastArticleClick: string, diffClickFirstPageHit: int]

In [ ]:
res.filter(f.length("sumArticleClick") > 0).count()

In [ ]:
res = res.withColumn("isBlogMainPage", f.when(
                                              (f.col("hitPagePath")=='/ru/s_m_business/pro_business/main/')|
                                              (f.col("hitPagePath")=='/ru/s_m_business/pro_business'),1
                                             ).otherwise(0)
                    )

# res = res.drop("visitNumber")
# res = res.withColumnRenamed('corr_visitNumber', 'visitNumber')

### Banners CTR exploration

In [ ]:
res = \
res.withColumn("showBanner",((f.col("hitPagePath") == "/ru/s_m_business")|
                             (f.col("hitPagePath") == "/ru/s_m_business/main/")|
                             (f.col("hitPagePath") == "/ru/s_m_business_alt")
                            )&
                            ((f.col('eventAction').like('%show_banner%'))
                            )
                   )\
    .withColumn("clickBanner",((f.col("hitPagePath") == "/ru/s_m_business")|
                                 (f.col("hitPagePath") == "/ru/s_m_business/main/")|
                                 (f.col("hitPagePath") == "/ru/s_m_business_alt")
                                )&
                                ((f.col('eventAction').like('%click_banner%'))|
                                 (f.lower(f.col('eventLabel')).like('%баннер%'))
                                )
                       )

In [ ]:
w=Window.partitionBy('cid').orderBy('sessionId') 

res = res.withColumn('next_state_m1', f.lead(f.col('hitPagePath'),count=1).over(w))\
         .withColumn('next_state_m2', f.lead(f.col('hitPagePath'),count=2).over(w)) 
    
res = res.withColumn('clickBannerAndGo2Landing',           
                                               ((f.col("hitPagePath") == "/ru/s_m_business")|
                                                (f.col("hitPagePath") == "/ru/s_m_business/main/")|
                                                (f.col("hitPagePath") == "/ru/s_m_business_alt")
                                               ) &
                                               (
                                                f.col('eventAction').like("%click_any%")|
                                                f.col('eventAction').like("%click_banner%")
                                               )&
                                               ((f.regexp_extract(f.col("next_state_m1"),
                                                                           "(?:/ru/s_m_business/)([a-zA-Z_/-]+)",1) != ''
                                                )|
                                                (f.regexp_extract(f.col("next_state_m2"),
                                                                           "(?:/ru/s_m_business/)([a-zA-Z_/-]+)",1) != ''
                                                )                                                
                                               )   
                    )\
         .withColumn("getProdNameFromLanding", f.when((f.col("clickBannerAndGo2Landing")==True)& 
                                                      (f.regexp_extract(f.col("next_state_m1"),
                                                                        "(?:/ru/s_m_business/)([a-zA-Z_/-]+)",1) !=''),
                                                       f.regexp_extract(f.col("next_state_m1"),
                                                                        "(?:/ru/s_m_business/)([a-zA-Z_/-]+)",1)                                                     
                                                     ) \
                                                .when((f.col("clickBannerAndGo2Landing")==True)& 
                                                      (f.regexp_extract(f.col("next_state_m2"),
                                                                        "(?:/ru/s_m_business/)([a-zA-Z_/-]+)",1) !=''),
                                                       f.regexp_extract(f.col("next_state_m2"),
                                                                        "(?:/ru/s_m_business/)([a-zA-Z_/-]+)",1)                                                     
                                                     ) \
                                                .otherwise(None)
                    )
   

res = res.withColumn('sumClickBannerAndGo2Landing', f.sum(f.col("clickBannerAndGo2Landing").cast(IntegerType())).over(w)
                    )

# res = res.withColumn("firstClickBanner",f.min(f.when((f.col("clickBannerAndGo2Landing")==True), f.col("sessionTime").cast(TimestampType())
#                                     ).otherwise()
#                              ).over(w)
#                     )     
    
# res = res.withColumn("",f.min(f.when((f.col("clickBannerAndGo2Landing")==True), f.col("getProdNameFromLanding"))
#                                     ).otherwise()
#                              ).over(w)
#                     )    

In [ ]:
w1=Window.orderBy('cid', 'sessionId', 'row_num')
w2=Window.partitionBy('cid', 'sessionId').orderBy('sessionId')

res = res.withColumn('row_num', f.row_number().over(w2))\
                           .withColumn('marker_col', f.when(f.col('clickBannerAndGo2Landing')==True,f.col('row_num')).otherwise(None))\
                           .withColumn('last', f.last('marker_col',True).over(Window.partitionBy('cid', 'sessionId')\
                                                                                    .orderBy('row_num').rowsBetween(Window.unboundedPreceding, Window.currentRow)))\

res=\
res.withColumn('visitBannerNumber', f.when(((f.lead(f.col('clickBannerAndGo2Landing'),count=1).over(w1)==True)&\
                                               (f.lead(f.col('clickBannerAndGo2Landing'),count=2).over(w1)==True)
                                              )&(f.col('clickBannerAndGo2Landing')!=False)
                                                   ,f.lead(f.col('last'),count=2).over(w1)
                                             )\
                                             .when(((f.lead(f.col('clickBannerAndGo2Landing'),count=1).over(w1)==True)&\
                                                    (f.lead(f.col('clickBannerAndGo2Landing'),count=2).over(w1)!=True)
                                                   )&(f.col('clickBannerAndGo2Landing')!=False)
                                                   ,f.lead(f.col('last'),count=1).over(w1)
                                                  )\
                                             .otherwise(f.col('last'))
                       )

res = res.drop(*["row_num", "marker_col", "last", ])

In [ ]:
cols = \
[ 
 'cid',
 'sessionId',
 'hitType',
 'url_path',
 'hitPagePath',
 'hitPageTitle',
 'isBlogMainPage',
 'interestedEvents', 
 'sessionStartTime',
 'hitNumber',
 'visitNumber',
 'visitBannerNumber', 
 'eventCategory',
 'eventAction',
 'eventLabel',
 'sessionTime',
 'hitText',
 'hitElemType',
 'hitTextStyle',
 'prodExtractedFromTitle',
 'countPageHit',
 'countEventHit',
 'sumXXSecOnPage',
 'sumScrollOnPage',
 'sessionRead', 
 'firstPageHit',
 'firstPageHitUrl',
 'lastPageHit',
 'firstEventHit',
 'lastEventHit',
 'firstArticleClick',
 'lastArticleClick',
 'diffClickFirstPageHit',
 'getCategory',
 'getArticleName',
 'articleTitles',
 'articleClick',
 'sumArticleClick',
 'extractedTextFromClick',
 'extractedText',
 'extractedEvents',
 'concatText',
 "showBanner",
 "clickBanner",
 'clickBannerAndGo2Landing',
 "sumClickBannerAndGo2Landing",   
 "getProdNameFromLanding" 
]


res = res.select(*cols)

# res.filter("articleTitles is not Null").show(100)

## Export to Hive 

In [21]:
# subprocess.call(['hdfs', 'dfs', '-rm', '-r', '-f', '-skipTrash', "hdfs://clsklsbx/user/team/team_digitcamp/hive/{}".format("recsys_ga_visit_fe_part_ext")])

0

In [ ]:
res.registerTempTable("tmp_visit")
hive.cacheTable("tmp_visit")
hive.sql("create table {}.{} select * from tmp_visit".format("sbx_team_digitcamp", "recsys_ga_visit_fe_part_ext_2"))

hive.sql("drop table if exists {}.{} purge".format("sbx_team_digitcamp", "recsys_ga_visit_fe_part_ext"))
subprocess.call(['hdfs', 'dfs', '-rm', '-r', '-f', '-skipTrash', "hdfs://clsklsbx/user/team/team_digitcamp/hive/{}".format("recsys_ga_visit_fe_part_ext")])

# hive.sql("insert into table {}.{} select * from tmp_visit".format("sbx_team_digitcamp", "recsys_ga_visit_fe_part_ext"))
hive.sql("alter table {tbl}.recsys_ga_visit_fe_part_ext_2 rename to {tbl}.recsys_ga_visit_fe_part_ext".format(tbl="sbx_team_digitcamp"))

# Check Final Table

In [59]:
res.filter("sumClickBannerAndGo2Landing > 1")\
   .select(  "hitPagePath",
             "hitPageTitle",
             'eventCategory',
             'eventAction',
             'eventLabel',
             'next_state_m1',
             'next_state_m2',
             "getProdNameFromLanding",
             'clickBannerAndGo2Landing',
             'visitBannerNumber'
          ).show(70)

,hitPagePath,hitPageTitle,eventCategory,eventAction,eventLabel,next_state_m1,next_state_m2,getProdNameFromLanding,clickBannerAndGo2Landing,visitBannerNumber
0,/ru/s_m_business,«Сбербанк» - Малый бизнес,None,None,None,/ru/s_m_business,/ru/s_m_business,None,False,129
1,/ru/s_m_business,«Сбербанк» - Малый бизнес,SITE_Corporate_.ru.s_m_business,show_banner,1_Антикризисный счёт для бизнеса от 0 ₽,/ru/s_m_business,/ru/s_m_business,None,False,129
2,/ru/s_m_business,«Сбербанк» - Малый бизнес,SITE_Corporate_.ru.s_m_business,informing_leads,30 sec on page,/ru/s_m_business,/ru/s_m_business,None,False,129
3,/ru/s_m_business,«Сбербанк» - Малый бизнес,SITE_Corporate_.ru.s_m_business,informing_leads,scroll 50 percent,/ru/s_m_business,/ru/s_m_business/bankingservice/acquiring_total,None,False,129
4,/ru/s_m_business,«Сбербанк» - Малый бизнес,SITE_Corporate_.ru.s_m_business,click_any,undefined_quick-services__image,/ru/s_m_business/bankingservice/acquiring_total,/ru/s_m_business/bankingservice/acquiring_total,bankingservice/acquiring_total,True,191
...,...,...,...,...,...,...,...,...,...,...
65,/ru/s_m_business/help/evotor,«Сбербанк» - Что такое онлайн-касса и для чего...,SITE_Corporate_help.evotor,informing_leads,scroll 50 percent,/ru/s_m_business/help/evotor,/ru/s_m_business/help/evotor,None,False,191
66,/ru/s_m_business/help/evotor,«Сбербанк» - Что такое онлайн-касса и для чего...,SITE_Corporate_help.evotor,informing_leads,scroll 75 percent,/ru/s_m_business/help/evotor,/ru/s_m_business/help/evotor,None,False,191
67,/ru/s_m_business/help/evotor,«Сбербанк» - Что такое онлайн-касса и для чего...,SITE_Corporate_help.evotor,click_any,Онлайн-касса «Эвотор»_footer__menu_link,/ru/s_m_business/help/evotor,/ru/s_m_business/onlinecredit?yclid=1925406541...,None,False,191
68,/ru/s_m_business/help/evotor,«Сбербанк» - Что такое онлайн-касса и для чего...,SITE_Widget_SiteFooter,reach_menu,Онлайн-касса Эвотор,/ru/s_m_business/onlinecredit?yclid=1925406541...,/ru/s_m_business/bankingservice/remoteservice/...,None,False,191


In [68]:
res = hive.table("{}.{}".format("sbx_team_digitcamp", "recsys_ga_visit_fe_part_ext"))

In [69]:
res.filter("clickBanner = True").select("eventLabel").distinct().count()

273

In [26]:
res.filter(f.col("eventLabel").like("%Банковская гарантия за 2 часа%") & f.col("eventAction").like("%click%"))\
   .select( "hitPagePath",
             "hitPageTitle",
             'eventCategory',
             'eventAction',
             'eventLabel',
             "showBanner",
             "clickBanner"
          ).show(30)

,hitPagePath,hitPageTitle,eventCategory,eventAction,eventLabel,showBanner,clickBanner
0,/ru/s_m_business,«Сбербанк» - Малый бизнес,SITE_Corporate_.ru.s_m_business,click_banner,6_Банковская гарантия за 2 часа,False,True
1,/ru/s_m_business,«Сбербанк» - Малый бизнес,SITE_Corporate_.ru.s_m_business,click_banner,3_Банковская гарантия за 2 часа,False,True
2,/ru/s_m_business,«Сбербанк» - Малый бизнес,SITE_Corporate_.ru.s_m_business,click_banner,5_Банковская гарантия за 2 часа,False,True
3,/ru/s_m_business,«Сбербанк» - Малый бизнес,SITE_Corporate_.ru.s_m_business,click_banner,5_Банковская гарантия за 2 часа,False,True
4,/ru/s_m_business,«Сбербанк» - Малый бизнес,SITE_Corporate_.ru.s_m_business,click_banner,5_Банковская гарантия за 2 часа,False,True
5,/ru/s_m_business,«Сбербанк» - Малый бизнес,SITE_Corporate_.ru.s_m_business,click_banner,6_Банковская гарантия за 2 часа,False,True
6,/ru/s_m_business,«Сбербанк» - Малый бизнес,SITE_Corporate_.ru.s_m_business,click_banner,6_Банковская гарантия за 2 часа,False,True
7,/ru/s_m_business,«Сбербанк» - Малый бизнес,SITE_Corporate_.ru.s_m_business,click_banner,6_Банковская гарантия за 2 часа,False,True
8,/ru/s_m_business,«Сбербанк» - Малый бизнес,SITE_Corporate_.ru.s_m_business,click_banner,6_Банковская гарантия за 2 часа,False,True
9,/ru/s_m_business,«Сбербанк» - Малый бизнес,SITE_Corporate_.ru.s_m_business,click_banner,5_Банковская гарантия за 2 часа,False,True


In [16]:
res.filter("articleClick = 1").select("cid").distinct().count()

10542

In [81]:
res.filter("getArticleName != '' ").select("cid").distinct().count()

584569